In [1]:
import pandas as pd 
import requests 
from plyer import notification
from datetime import datetime
import notificacao
import sqlite3
import re
import funcoes as func


# Extraindo a tabela banks da API
url = "https://brasilapi.com.br/api/banks/v1"

bancos = func.importar(url)

data = func.formato_datas()

bancos_json = func.salvar_tabela(bancos)

# Transformando as tabelas em um DataFrame
bancos_full = pd.DataFrame(bancos_json)

# Excluindo linhas com a coluna code que possuem missing values
bancos_full = bancos_full.dropna(subset=['code'])

#Transformando os dados da coluna code de float para int
bancos_full = bancos_full.astype({'code':int})

# Conectando com banco de dados e salvando os dados brutos
bancos_connect = sqlite3.connect('Projeto Final.db')
bancos_full.to_sql('Bancos Bruto', bancos_connect, if_exists='replace', index=False )

# Alterando o nome das colunas
bancos_full_renomeado = func.altera_colunas(bancos_full)

# Alterando a coluna Nome Completo para deixar todos nomes em maiusculo
bancos_full_renomeado['Nome Completo']=bancos_full_renomeado['Nome Completo'].str.upper()

notificacao.alerta(bancos, data, url)

#Tratamento de Missing Values
bancos_full_tratado = (bancos_full_renomeado.dropna())

# Ordenação pela coluna Código
bancos_full_tratado.sort_values('Código')

# Salvando os dados tratados no banco de dados e fechando a conexão
bancos_full_tratado.to_sql('Bancos Tratado', bancos_connect, if_exists='replace', index=False )
bancos_connect.close()


bancos_full_tratado



,ISPB,Nome,Código,Nome Completo
0,00000000,BCO DO BRASIL S.A.,1,BANCO DO BRASIL S.A.
1,00000208,BRB - BCO DE BRASILIA S.A.,70,BRB - BANCO DE BRASILIA S.A.
4,00122327,SANTINVEST S.A. - CFI,539,"SANTINVEST S.A. - CREDITO, FINANCIAMENTO E INV..."
5,00204963,CCR SEARA,430,COOPERATIVA DE CREDITO RURAL SEARA - CREDISEARA
6,00250699,AGK CC S.A.,272,AGK CORRETORA DE CAMBIO S.A.
...,...,...,...,...
352,92856905,ADVANCED CC LTDA,117,ADVANCED CORRETORA DE CÂMBIO LTDA
353,92874270,BCO DIGIMAIS S.A.,654,BANCO DIGIMAIS S.A.
354,92875780,WARREN CVMC LTDA,371,WARREN CORRETORA DE VALORES MOBILIÁRIOS E CÂMB...
355,92894922,BANCO ORIGINAL,212,BANCO ORIGINAL S.A.


In [2]:
# Extraindo a tabela corretoras da API
url = "https://brasilapi.com.br/api/cvm/corretoras/v1"

corretoras = func.importar(url)

data = func.formato_datas()


corretoras_json = func.salvar_tabela(corretoras)

# Transformando as tabelas em um DataFrame
corretoras_full = pd.DataFrame(corretoras_json)

# Conectando com banco de dados e salvando os dados brutos
corretoras_connect = sqlite3.connect('Projeto Final.db')
corretoras_full.to_sql('Corretoras Bruto', corretoras_connect, if_exists='replace', index=False )

# Alterando o nome das colunas

corretoras_full_renomeado = func.nome_colunas(corretoras_full)

# Notificacao de importacao dos dados

notificacao.alerta(corretoras, data, url)

# Formatacao da coluna valor patrimonio

func.formato_valor(corretoras_full_renomeado,'Valor Patrimonio')

# Formatacao das colunas com datas

func.formato_data(corretoras_full_renomeado,'Data Registro')

# Exclusão de colunas irrelevantes para a análise. 

corretoras_full_colunas = func.exclusao_colunas(corretoras_full_renomeado)

# Ajustando o Formato do CNPJ

corretoras_full_colunas['CNPJ'] = func.formato_cnpj(corretoras_full_colunas,'CNPJ')

# Ajustando Formato do Telefone
filtro_tel =  func.filtro_str_len(corretoras_full_colunas,'Telefone',8) 
corretoras_full_colunas.loc[filtro_tel, 'Telefone'] = func.ajuste_telefone(corretoras_full_colunas,filtro_tel,'Telefone')
corretoras_full_colunas['Telefone'] = func.formato_telefone(corretoras_full_colunas, 'Telefone')

# Excluindo corretoras que não possuem Telefone na base de dados
filtro_sem_tel = func.string_vazia(corretoras_full_colunas,'Telefone',1) 
corretoras_full_colunas = func.drop_linhas(corretoras_full_colunas, filtro_sem_tel)

#Excluindo Corretoras que não possuem Pais na base de dados
filtro_pais = func.string_vazia(corretoras_full_colunas,'Pais',1)
corretoras_full_colunas = func.drop_linhas(corretoras_full_colunas,filtro_pais)

#Excluindo Corretoras que não possuem Municipio na base de dados
filtro_municipio = func.string_vazia(corretoras_full_colunas, 'Municipio', 0)
corretoras_full_colunas = func.drop_linhas(corretoras_full_colunas,filtro_municipio)

#Excluindo Corretoras que não possuem UF na base de dados
filtro_UF = func.string_vazia(corretoras_full_colunas,'UF',0)
corretoras_full_colunas = func.drop_linhas(corretoras_full_colunas,filtro_UF) 

#Excluindo Corretoras que não possuem Bairro na base de dados
filtro_bairro = func.string_vazia(corretoras_full_colunas,'Bairro',0)
corretoras_full_colunas = func.drop_linhas(corretoras_full_colunas,filtro_bairro)

#Excluindo corretoras que não possuem nome comercial
filtro_nome_comercial = func.string_vazia(corretoras_full_colunas,'Nome Comercial',0)
filtro_nome = func.caracter_especial(corretoras_full_colunas,'Nome Comercial','--')
corretoras_full_colunas = func.drop_linhas(corretoras_full_colunas,filtro_nome_comercial)
corretoras_full_colunas = func.drop_linhas(corretoras_full_colunas,filtro_nome)

#Excluindo corretoras que possuem valor de patrimonio zerado
filtro_valor = func.caracter_especial(corretoras_full_colunas,'Valor Patrimonio', 'R$0.00')
corretoras_full_colunas = func.drop_linhas(corretoras_full_colunas,filtro_valor)

# Filtrando as corretoras que estão em funcionamento
corretoras_ativas = corretoras_full_colunas.loc[corretoras_full_colunas['Status'] =='EM FUNCIONAMENTO NORMAL']

# Filtrando as corretoras que estão canceladas
corretoras_canceladas = corretoras_full_colunas.loc[corretoras_full_colunas['Status'] =='CANCELADA']

# Salvando os dados tratados no banco de dados e fechando a conexão
corretoras_full_colunas.to_sql('Corretoras Tratado', corretoras_connect, if_exists='replace', index=False )
corretoras_ativas.to_sql('Corretoras Tratado Ativas', corretoras_connect, if_exists='replace', index=False )
corretoras_canceladas.to_sql('Corretoras Tratado Canceladas', corretoras_connect, if_exists='replace', index=False )
bancos_connect.close()

corretoras_full_colunas


,CNPJ,Razão Social,Nome Comercial,Status,Telefone,Pais,UF,Municipio,Bairro,Valor Patrimonio,Codigo CVM,Data Registro
0,76.621.457/0001-85,4UM DTVM S.A.,4UM INVESTIMENTOS,CANCELADA,3351-9966,BRASIL,PR,CURITIBA,CENTRO,"R$4,228,660.18",2275,15-01-1968
4,33.819.590/0001-38,AÇÃO S.A. CORRETORA DE VALORES E CÂMBIO,ACAO SA CVC,CANCELADA,3509-5755,BRASIL,RJ,RIO DE JANEIRO,CENTRO,"R$291,862.38",1112,19-04-1968
5,92.856.905/0001-86,ACTION CCVM S.A.,ACTION CORRETORA,CANCELADA,3228-7977,BRASIL,SP,SÃO PAULO,JARDIM PAULISTA,"R$666,907.71",2500,22-05-1967
6,24.159.923/0001-59,ACTION S.A. EMPREENDIMENTOS E PARTICIPAÇÕES,ACTION DISTRIBUIDORA,CANCELADA,3178-8923,BRASIL,SP,SÃO PAULO,JARDIM PAULISTANO,"R$629,349.55",736,06-10-1988
12,74.014.747/0001-35,AGORA CTVM S.A.,AGORA-SENIOR CTVM S.A .,EM FUNCIONAMENTO NORMAL,2178-6198,,SP,SÃO PAULO,BELA VISTA,"R$449,810,368.63",2895,22-12-1993
...,...,...,...,...,...,...,...,...,...,...,...,...
369,01.170.892/0001-31,VOTORANTIM CTVM LTDA,VOTORANTIM,CANCELADA,5171-1894,BRASIL,SP,SÃO PAULO,VL.GERTRUDES,"R$266,791,832.77",3166,15-03-1996
370,61.769.790/0001-69,WALPIRES S/A CCTVM,WALPIRES S/A CCTVM,CANCELADA,2808-7065,BRASIL,SP,SÃO PAULO,JARDIM PAULISTANO,"R$7,464,373.90",1902,20-09-1967
371,92.875.780/0001-31,WARREN CORRETORA DE VALORES MOBILIÁRIOS E CÂMB...,WARREN CORRETORA,EM FUNCIONAMENTO NORMAL,3237-2273,BRASIL,RS,PORTO ALEGRE,BOM FIM,"R$23,398,120.52",2534,06-12-1967
372,24.543.688/0001-14,WARREN CORRETORA DE VALORES MOBILIÁRIOS E CÂMB...,PERFIL,CANCELADA,3224-9022,BRASIL,PE,RECIFE,CASA FORTE,"R$608,260.02",2658,14-11-1989


In [3]:

# Extraindo a tabela PIX da API
url = 'https://brasilapi.com.br/api/pix/v1/participants'

pix = requests.get(url)

data = datetime.now().strftime("%d/%m/%Y %H:%M:%S")


pix_json = func.salvar_tabela(pix)

# Transformando as tabelas em um DataFrame
pix_full = pd.DataFrame(pix_json)

# Conectando com banco de dados e salvando os dados brutos
pix_connect = sqlite3.connect('Projeto Final.db')
pix_full.to_sql('Pix Bruto', pix_connect, if_exists='replace', index=False )

# Alterando o nome das colunas
pix_full_renomeado = func.modifica_coluna(pix_full)

notificacao.alerta(pix, data, url)

# Tratamento da coluna Inicio que contem data e hora
pix_full_renomeado['Inicio'] = pd.to_datetime(pix_full_renomeado['Inicio'])
pix_full_renomeado['Inicio'] = func.ajusta_datahora(pix_full_renomeado,'Inicio')

# Salvando os dados tratados no banco de dados e fechando a conexão
pix_full_renomeado.to_sql('Pix Tratado', pix_connect, if_exists='replace', index=False )
pix_connect.close()


pix_full_renomeado


,ISPB,Nome,Nome Reduzido,Modalidade,Tipo,Inicio
0,00000000,BANCO DO BRASIL S.A.,BCO DO BRASIL S.A.,PDCT,DRCT,03-11-2020 09:30
1,00000208,BRB - BANCO DE BRASILIA S.A.,BRB - BCO DE BRASILIA S.A.,PDCT,DRCT,03-11-2020 09:30
2,00068987,COOPERATIVA DE CRÉDITO ARACREDI LTDA. - SICOOB...,CC ARACREDI LTDA.,PDCT,IDRT,03-11-2020 09:30
3,00075847,COOPERATIVA DE CRÉDITO UNICRED CENTRO-SUL LTDA...,CC UNICRED CENTRO-SUL LTDA - UNICRED CENTRO-SUL,PDCT,IDRT,03-11-2020 09:30
4,00106180,CENTRAL DAS COOPERATIVAS DE CRÉDITO DOS ESTADO...,"CCC DOS EST DE MT, MS E MUN DE CACOAL/RO",PDCT,IDRT,03-11-2020 09:30
...,...,...,...,...,...,...
799,95213211,"COOPERATIVA DE CRéDITO, POUPANçA E INVESTIMENT...",COOP SICREDI CAMINHO DAS ÁGUAS RS,PDCT,IDRT,03-11-2020 09:30
800,95424891,"COOPERATIVA DE CRÉDITO, POUPANÇA E INVESTIMENT...",COOP SICREDI VALE DO RIO PARDO,PDCT,IDRT,03-11-2020 09:30
801,95594941,"COOPERATIVA DE CRÉDITO, POUPANÇA E INVESTIMENT...",COOP SICREDI REG CENTRO RS/MG,PDCT,IDRT,03-11-2020 09:30
802,97259253,COOPERATIVA E ECONOMIA E CRÉDITO MÚTUO DOS MÉD...,CECM MÉDICOS PLANALTO MÉDIO RS - CREDISIS/CRED...,PDCT,IDRT,04-10-2021 12:53
